**MAKEMORE: An autoregressive character-level language model**

We will use **Bigrams and MLP** in this notebook.

Name: **Lakshit Sethia**

Roll No.: **210102123**

DA623: Final Project

In this we are basically going to do next-prediction based on characters. So for each letter, we will be providing a large dataset of names of people, and it will be able to generate many more related names.





**Reading and exploring the dataset**

I have created a dataset containing Indian Names by combining and refining multiple datasets. The final data is in the file ***names.txt***.

[1] www.kaggle.com/datasets/ananysharma/indian-names-dataset

[2] www.kaggle.com/datasets/jasleensondhi/indian-names-corpus-nltk-data

[3] www.kaggle.com/datasets/shubhamuttam/indian-names-by-gender


In [ ]:
words = open('names.txt', 'r').read().splitlines()

In [ ]:
words[:10]

In [ ]:
len(words)

In [ ]:
min(len(word) for word in words)

In [ ]:
max(len(word) for word in words)

**Exploring the bigrams in the dataset**

Two columns are formed where The first character is taken and is zipped with its next one. So when there is only one character left, then it exits.




In [ ]:
for word in words[:1]:  #Example
    for ch1, ch2 in zip(word, word[1:]):
        print(ch1, ch2)

Here we are just adding our own set of characters in the list, so as to get like this custom output - Labelling the first and the last character of the word

In [ ]:
for word in words[:1]:
    chs = ['<S>'] + list(word) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        print(ch1, ch2)

In [ ]:
for word in words[:3]:
    chs = ['<S>'] + list(word) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        print(ch1, ch2)

**Counting bigrams in a python dictionary**

We have created a dictionary 'b' - First data structure.

Then we have created a set 'bigram' which is just a set of two characters - Second data structure.

Here we are adding it to the dictionary 'b', where the key is 'bigram' (which is a set of character pairs) has the value counts of the set occoured (The number of times that set has occured).

In [ ]:
b = {}  #dictionary 'b'
for word in words[:1]:
    chs = ['<S>'] + list(word) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2) #bigram
        b[bigram] = b.get(bigram, 0) + 1
        print(ch1, ch2)

**Note:**   \
`b.get(bigram)` is the same as `b[bigram]`  \
Just that here: `b.get(bigram, 0)` if we don't get a bigram value, we want it to assign to 0    \
Finally we are adding one `+1` as we want to count the occurance.

In [ ]:
b

In [ ]:
b = {}
for word in words[:3]:
    chs = ['<S>'] + list(word) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1
        print(ch1, ch2)

In [ ]:
b

In [ ]:
b = {}
for word in words:
    chs = ['<S>'] + list(word) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1

# b

We know 'b' is a dictionary. So .items() basically gives us that it's values in a (Key, Value) set

In [ ]:
b.items()

Now this by default sorts the values based on the Key

In [ ]:
sorted(b.items())

 Now, here we are specifying we want to sort based on the values. So we select the key, then in the lambda function, we take the keyvalue (kv) and select the second element in the set, which is what we want
 - sign is for descending

In [ ]:
sorted(b.items(), key= lambda kv: -kv[1])

**Counting bigrams in a 2D torch tensor ("training the model")**

In [ ]:
import torch

In [ ]:
N = torch.zeros((28, 28), dtype = torch.int32)

In [ ]:
chars = sorted(list(set(''.join(words))))

stoi = {s:i for i,s in enumerate(chars)}
stoi['<S>'] = 26
stoi['<E>'] = 27

In [ ]:
stoi

In [ ]:
for word in words:
    chs = ['<S>'] + list(word) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1

In [ ]:
N

**Visualizing the bigram tensor**

In [ ]:
itos = {i:s for s,i in stoi.items()}
itos

Bigram Plot

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(28):
    for j in range(28):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off');

**Deleting spurious (S) and (E) tokens in favor of a single . token**

In [ ]:
N = torch.zeros((27, 27), dtype = torch.int32)

chars = sorted(list(set(''.join(words))))

stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0

In [ ]:
itos = {i:s for s,i in stoi.items()}

In [ ]:
for word in words:
    chs = ['.'] + list(word) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off');

**Sampling from the model**

In [ ]:
N[0]    #Viewing just the first row

First we make them all into float. \
Then we make a probability distribution \
We do that by dividing `p` with `p.sum()`

In [ ]:
p = N[0].float()
p = p / p.sum()
p

In [ ]:
p.sum().item()

So the total probability sums up to 1. Therefore now we have the probability values for each of those characters.

In [ ]:
g = torch.Generator().manual_seed(210102123)
p = torch.rand(3, generator=g)
p = p / p.sum()
p

In [ ]:
torch.multinomial(p, num_samples=20, replacement=True, generator=g)

So based on the probability percentages `p` we get a bunch of sample values \
So `0` should be 60%, `1` should be 30%, `2` should be 10% of the total samples generated

In [ ]:
g = torch.Generator().manual_seed(210102123)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itos[ix]

So now, if we had got another sampled value, lets say 'm' (so we have taken the column), now we go to the row containing 'm' and then check for its correspondant character.    \
\
Keeping this jest in mind, we will be making this into a loop.

In [ ]:
g = torch.Generator().manual_seed(210102123)

for i in range(10):
    out = []
    ix = 0
    while True:
        p = N[ix].float()
        p = p / p.sum()
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

And this is why the Bigram model is so bad lol. The output generated not to great (Andrej said "terrible" xd), for example for `p.` , is that the model doesn't understand that 'p' should have had something before or after it. Right now, it considers that as a name itself.

But now we will see why the output made by the model is not exactly too terrible

In [ ]:
g = torch.Generator().manual_seed(210102123)

for i in range(10):
    out = []
    ix = 0
    while True:
        # p = N[ix].float()
        # p = p / p.sum()
        p = torch.ones(27) / 27.0
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

So now this is what we get when the model is completely untrained, it gives you a garbage of values.

This is happening because we removed the probability distribution and added a distribution of uniform values. So all of the characters are equally likely to occur.

So yeah, if we train it with a Bigram, then its a lot better output. So ultimately it is actually working, just that Bigram is not so great for this.

In [ ]:
#Solving the inefficiency problem
P = N.float()
P = P / P.sum(1, keepdim=True) #Here is where we are applying the sum and broadcasting rules. Sum for the function and Broadcasting for the division part that takes place.

# 27 27
# 27  1

In [ ]:
P[0].sum() #This should return the tensor object with value 1. So that entire row as been normalised

So the rule says:
Two tensors are broadcastable if the following rules hold:

Each tensor has at least one dimension.

When iterating over the dimension sizes, starting at the trailing dimension, the dimension sizes must either be equal, one of them is 1, or one of them does not exist.

So in our case, one of the dimentional sizes is one i.e 27 **1** \
And, the dimension sizes are equal: \
**27**  \
**27**




In [ ]:
g = torch.Generator().manual_seed(210102123)

for i in range(10):
    out = []
    ix = 0
    while True:
        p = P[ix]
        # p = N[ix].float()
        # p = p / p.sum()
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

In [ ]:
P = N.float()
P /= P.sum(1, keepdim=True)

In [ ]:
g = torch.Generator().manual_seed(210102123)

for i in range(10):
    out = []
    ix = 0
    while True:
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

--------------

**Evaluating our model**

In [ ]:
import torch

In [ ]:
N = torch.zeros((27, 27), dtype=torch.int32)

chars = sorted(list(set(''.join(words))))

stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0

In [ ]:
itos = {i:s for s,i in stoi.items()}

In [ ]:
for word in words:
  chs = ['.'] + list(word) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1, ix2] += 1

In [ ]:
P = N.float()
P /= P.sum(1, keepdim=True)

**Likelihood**: The probability of observing the data given the model.

**Log-likelihood**: The natural logarithm of the likelihood.

**Negative Log-likelihood (NLL)**: The negative of the log-likelihood

In [ ]:
log_likelihood = 0.0
n = 0

for word in words[:3]:
    chs = ['.'] + list(word) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2] #Likelihood - product of all the values
        logprob = torch.log(prob) #Log Likelihood
        log_likelihood += logprob #Log Likelihood - Adding the logs of all probability values
        n += 1 #Log Likelihood - For the average
        print(f'{ch1}{ch2}: {prob:.4f} {logprob: .4f}')

print(f'{log_likelihood=}')
nll = -log_likelihood #Negative Log Likelihood
print(f'{nll=}')
print(f'{nll/n}') #Negative Log Likelihood - Average value


Model smoothening

In [ ]:
P = (N+1).float()
P /= P.sum(1, keepdim=True)

In [ ]:
log_likelihood = 0.0
n = 0

# for word in words[:3]:
for word in ["lakshit"]:
    chs = ['.'] + list(word) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2] #Likelihood - product of all the values
        logprob = torch.log(prob) #Log Likelihood
        log_likelihood += logprob #Log Likelihood - Adding the logs of all probability values
        n += 1 #Log Likelihood - For the average
        print(f'{ch1}{ch2}: {prob:.4f} {logprob: .4f}')

print(f'{log_likelihood=}')
nll = -log_likelihood #Negative Log Likelihood
print(f'{nll=}')
print(f'{nll/n}') #Negative Log Likelihood - Average value


Here, we arrived at the model doing everything explicitly. We were performing counts and we were normalizing those counts.

Now, we'll be doing an alternative approach but the final output will be the same.

**Here we are going to cast the problem of Bigram Character level language modelling into a neural network**

So our NN will still be a character level language model.

So we have an input character -> given to the neural network and then it is gonna predict the probability -> of the next character that is likely to follow.

And in addition to that, we are going to be able to evaluate any setting of the parameters of the langauage model, because we have a loss function value (The NLL).

So we are going to look at the probability distributions and we are going to look at its labels (in the NN) which are basically the identity of the next character in the Bigram.

So knowing what character comes next is the bigram, allows us to check what will be the probability value assigned to that character (So higher the value, the better. Because it is another way of saying that the loss is low).

**We're gonna use gradient based optimization to tune the parameters of this network.**
Because we have a loss function and we're gonna minimize it.
We're gonna tune the weights, so that the NN is gonna correctly predict the next probability of the next characters.

In [ ]:
#Creating the training set of bigrams (x,y)
xs, ys = [], []

for word in words[:1]:
    chs = ['.'] + list(word) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        print(ch1, ch2)
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [ ]:
xs

In [ ]:
ys

Feeding these examples into a neural network

In [ ]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=27).float() #IMP: manual type casting
xenc

In [ ]:
xenc.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(xenc)

In [ ]:
W = torch.randn((27, 27))   #Generating the weights
xenc @ W    #Doing matrix multiplication

In [ ]:
logits = xenc @ W   #log-counts
counts = logits.exp()   #equivalent to N matrix as before
probs = counts / counts.sum(1, keepdims=True)   #Normalising the rows (To calculate the probability)
probs

-------------

In [ ]:
xs

In [ ]:
ys

Randomly initialize 27 neurons' weights. each neuron receives 27 inputs

In [ ]:
g = torch.Generator().manual_seed(210102123)
W = torch.randn((27, 27), generator=g)

Input to the network: one-hot encoding

In [ ]:
xenc = F.one_hot(xs, num_classes=27).float()
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character

the last 2 lines here are together called a 'softmax'

In [ ]:
probs.shape

In [ ]:
nlls = torch.zeros(5)
for i in range(5):
  # i-th bigram:
  x = xs[i].item() # input character index
  y = ys[i].item() # label character index
  print('--------')
  print(f'bigram example {i+1}: {itos[x]}{itos[y]} (indexes {x},{y})')
  print('input to the neural net:', x)
  print('output probabilities from the neural net:', probs[i])
  print('label (actual next character):', y)
  p = probs[i, y]
  print('probability assigned by the net to the the correct character:', p.item())
  logp = torch.log(p)
  print('log likelihood:', logp.item())
  nll = -logp
  print('negative log likelihood:', nll.item())
  nlls[i] = nll

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())

In [ ]:
import torch.nn.functional as F

**OPTIMIZATION**

In [ ]:
xs

In [ ]:
ys

Adding the third parameter here (requires_grad=True) for the Backward pass

In [ ]:
g = torch.Generator().manual_seed(210102123)
W = torch.randn((27, 27), generator=g, requires_grad=True)

FORWARD PASS

In [ ]:
xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
loss = -probs[torch.arange(5), ys].log().mean() #torch.arange(5) is basically 0 to 5(4) position, ys is from that tuple list | We calculate the probability values of that | Then we take their log values | Then we take their mean | Finally take the negative value (since NLL)

In [ ]:
loss

BACKWARD PASS

In [ ]:
W.grad = None
loss.backward()

In [ ]:
W.grad.shape

In [ ]:
W.grad

UPDATE


In [ ]:
W.data += -0.1 * W.grad

JUST PUTTING THEM TOGETHER TO PERFORM GRADIENT DESCENT

In [ ]:
#ONLY RUN THIS THE FIRST TIME
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(210102123)
W = torch.randn((27, 27), generator=g, requires_grad=True) #Adding the third parameter here for the Backward pass (as remember in micrograd we had done the same thing)

In [ ]:
#FORWARD PASS
xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
loss = -probs[torch.arange(5), ys].log().mean() #torch.arange(5) is basically 0 to 5(4) position, ys is from that tuple list | We calculate the probability values of that | Then we take their log values | Then we take their mean | Finally take the negative value (since NLL)

In [ ]:
print(loss.item()) #CHECKING THE LOSS VALUE

In [ ]:
#BACKWARD PASS
W.grad = None #the gradient is first set to zero
loss.backward()

In [ ]:
#UPDATE
W.data += -0.1 * W.grad

**PUTTING THEM ALL TOGETHER**

In [ ]:
# create the dataset
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(210102123)
W = torch.randn((27, 27), generator=g, requires_grad=True)

In [ ]:
# gradient descent
for k in range(20):

  # forward pass
  xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())

  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()

  # update
  W.data += -50 * W.grad

SO WE ALMOST ACHIEVED A VERY LOW LOSS VALUE. SIMILAR TO THE LOSS VALUE WE CALCULATED WITHOUT NN, WHEN WE TYPED OUR OWN NAME AND SAW HOW IT PERFORMED.

Finally *drumrolls*, we are going to see how sampling from this model produces the outputs (Spoiler alert: it will be the same as how we made the model manually, coz... it is the same model just that we made it using Neural nets)

In [ ]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(210102123)

for i in range(5):

  out = []
  ix = 0
  while True:

    # ----------
    # BEFORE:
    #p = P[ix]
    # ----------
    # NOW:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------

    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))